## LSTM Example Notebook
This notebook provides several examples of how to use nn.LSTM. 

# Sequence Example
Based on the example here: https://pytorch.org/tutorials/beginner/nlp/sequence_models_tutorial.html

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

# Build the network

In [ ]:
lstm = nn.LSTM(3, 3, 2) # Input dim is 3, Output dim is 3
inputs = [torch.randn(1, 3) for _ in range(2)] # make a sequence of length 5
print(f'[INPUTS] shape: ({len(inputs)}, {inputs[0].shape})')
print(f'[INPUTS] elements: {inputs}')

# Notes:
- The first element of the hidden shape must be the same as the `num_layers` in the LSTM
- The third element of the hidden shape mush be the same as the `hidden_size` in the LSTM

In [ ]:
hidden = (torch.randn(2, 1, 3), 
          torch.randn(2, 1, 3))
print(f'[HIDDEN] shape: ({len(hidden)}, {hidden[0].shape})')
print(f'[HIDDEN] elements: {hidden}')

In [ ]:
for i in inputs:
    # step through the sequence on element at a time
    # after each step, hidden contains the hidden state
    print(f'[INPUT] {i.shape}, {i}')
    reshapped_i = i.view(1, 1, -1)
    print(f'[I VIEW] {reshapped_i.shape}, element: {reshapped_i}')
    out, hidden = lstm(reshapped_i, hidden)
    print(f'[OUT] {out.shape}, elements: {out}')
    print(f'[HIDDEN] ({len(hidden)},{hidden[0].shape}), elements: {hidden}')

In [ ]:
# Alternatively, we can do the entire sequence all at once
# The first value returned by the LSTM is all of the hidden states throughout
# the sequence. The second is just the most recent hidden state
# (compare the last slice of "out" with "hidden" below, then are the same)
# The reason for this is:
# "out" will give you access to all hidden states in the sequence, 
# "hidden" will allow you to continue the sequence and backpropogate,
# by passing it as an arguement to the LSTM at a later time

inputs = torch.cat(inputs).view(len(inputs), 1, -1)
print(f'[INPUTS] shape: {inputs.shape}')
hidden = (torch.randn(2, 1, 3), torch.randn(2, 1, 3)) # Clean out the hidden state
out, hidden = lstm(inputs, hidden)
print(f'[OUT] {out}')
print(f'[HIDDEN] {hidden}')

## LSTM for Part-of-Speach Tagging

In [ ]:
def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)

In [ ]:
training_data = [
    ("The dog ate the apple".split(), ["DET", "NN", "V", "DET", "NN"]),
    ("Everybod read that book".split(), ["NN", "V", "DET", "NN"])
]
print(training_data[0][0], training_data)

word_to_idx = {}

for sent, tags in training_data:
    for word in sent:
        if word not in word_to_idx:
            word_to_idx[word] = len(word_to_idx)
print(word_to_idx)
tag_to_idx = {"DET": 0, "NN":1, "V":2}


EMBEDDING_DIM = 6 # usually something like 32 or 64
HIDDEN_DIM = 6 # usually something like 32 or 64

In [ ]:
class LSTMTagger(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, vocab_size, target_size):
        super().__init__()
        self.hidden_dim = hidden_dim
        print(vocab_size, embedding_dim)
        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)
        print(self.word_embeddings)
        # the lstm takes word embeddings as input and outputs hidden state
        # with dimensionality hidden_dim
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = nn.Linear(hidden_dim, target_size)

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        print(sentence.shape)
        print(f'[EMBEDS] {embeds.shape}')
        transformed = embeds.view(len(sentence), 1, -1)
        print(transformed.shape, transformed)
        lstm_out, _ = self.lstm(transformed)
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

In [ ]:
model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_idx), len(tag_to_idx))
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr = 0.1)

with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_idx)
    print('[INPUTS]', inputs.shape, inputs)
    tag_score = model(inputs)
    print(tag_score)


In [ ]:
for epoch in range(300):
    for sentence, tag in training_data:
        model.zero_grad()
        sentence_in = prepare_sequence(sentence, word_to_idx)
        targets = prepare_sequence(tag, tag_to_idx)
        tag_scores = model(sentence_in)

        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()
        
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_idx)
    sentence = ""
    for i in inputs:
        for key, value in word_to_idx.items():
            if i == value:
                sentence += key+" "
    print(sentence)
    tag_scores = model(inputs)
    print(f'[INPUTS] {inputs}')
    predictions = torch.argmax(tag_scores, dim=1)
    tags = ""
    for p in predictions:
        for key, value in tag_to_idx.items():
            if p == value:
                tags += key + " "

    print(tags)
    print(f'[PREDICTIONS] {predictions}')
    print(tag_score)

## GSP RL LSTM example

In [5]:
from gsp_rl.src.networks import EnvironmentEncoder, DQN
import torch
import torch.nn
import numpy as np

lstm_nn_args = {
    'lr':1e-5,
    'input_size':5,
    'output_size':1,
    'embedding_size':256,
    'hidden_size':256,
    'num_layers':5,
    'batch_size':16
}
dqn_nn_args = {
    'id': 1, 
    'lr': 1e-5,
    'input_size': lstm_nn_args['output_size'],
    'output_size': 3,
}
lstm_actor = EnvironmentEncoder(**lstm_nn_args)
actor = DQN(**dqn_nn_args)
print(lstm_actor)

testing_data = [torch.randn((5)) for _ in range(10)]
testing_data = np.array(testing_data)
print(testing_data.shape)
out = lstm_actor(torch.tensor(testing_data))
actions = actor(out)

#print(out)
print('[ACTIONS]', actions)
# embedding  = nn.Linear(5, 10)
# sample_in = torch.randn(5)
# print(sample_in.shape, sample_in)
# out = embedding(sample_in)


EnvironmentEncoder(
  (embedding): Linear(in_features=5, out_features=256, bias=True)
  (ee): LSTM(256, 256, num_layers=5, batch_first=True)
  (meta_layer): Linear(in_features=256, out_features=1, bias=True)
)
(10, 5)
[ACTIONS] tensor([[[-0.0759, -0.0507, -0.0458]],

        [[-0.0759, -0.0507, -0.0458]],

        [[-0.0759, -0.0507, -0.0458]],

        [[-0.0759, -0.0507, -0.0458]],

        [[-0.0759, -0.0507, -0.0458]],

        [[-0.0759, -0.0507, -0.0458]],

        [[-0.0759, -0.0507, -0.0458]],

        [[-0.0759, -0.0507, -0.0458]],

        [[-0.0759, -0.0507, -0.0458]],

        [[-0.0759, -0.0507, -0.0458]]], grad_fn=<ViewBackward0>)


In [14]:
from gsp_rl.src.networks import EnvironmentEncoder, DDPGActorNetwork, DDPGCriticNetwork, RDDPGCriticNetwork, RDDPGActorNetwork
import torch
import numpy as np
lstm_nn_args = {
    'lr':1e-5,
    'input_size':5,
    'output_size':256,
    'embedding_size':200,
    'hidden_size':300,
    'num_layers':5,
    'batch_size':16
}

ddpg_actor_nn_args = {
    'id': 1,
    'lr': 1e-4,
    'input_size':lstm_nn_args['output_size'],
    'output_size': 2,
    'fc1_dims': 400,
    'fc2_dims': 300
}
ddpg_critic_nn_args = {
    'id': 1,
    'lr': 1e-4,
    'input_size':lstm_nn_args['output_size'] + ddpg_actor_nn_args['output_size'],
    'output_size': 2,
    'fc1_dims': 400,
    'fc2_dims': 300
}

def makeRDDPG(lstm_nn_args, ddpg_actor_nn_args, ddpg_critic_nn_args):
    ee = EnvironmentEncoder(**lstm_nn_args)
    actor = DDPGActorNetwork(**ddpg_actor_nn_args)
    critic = DDPGCriticNetwork(**ddpg_critic_nn_args)
    rddpg_actor = RDDPGActorNetwork(ee, actor)
    rddpg_critic = RDDPGCriticNetwork(ee, critic)
    return rddpg_actor, rddpg_critic

actor, critic = makeRDDPG(lstm_nn_args, ddpg_actor_nn_args, ddpg_critic_nn_args)
ee = EnvironmentEncoder(**lstm_nn_args)
testing_data = [torch.randn((lstm_nn_args['input_size'])) for _ in range(10)]
testing_data = torch.tensor(np.array(testing_data))
np_data = [np.random.randint(1, 25, lstm_nn_args['input_size']) for _ in range(10)]
np_data = np.array(np_data)
print(testing_data.shape, np_data.shape)
action = actor(testing_data)
# print(action)
# print(action.view(testing_data.shape[0], 2))
# print(testing_data)
torch.cat([testing_data, action.view(testing_data.shape[0], 2)], dim=-1)
# print(testing_data.shape, action.shape)

value = critic(testing_data, action)
# print(value)

# for name, param in ee.named_parameters():
#     print(name, param.shape)


SyntaxError: invalid syntax (663847573.py, line 41)